# Google Colab 実験ノートブック
このノートブックは、Google Colab 上で **世界モデルを用いたタグ付き協調ゲーム** の学習・解析コードを実行するためのハンズオン手順をまとめたものです。
リポジトリを取得して依存ライブラリを導入し、最小構成の自己対戦実験を再現する流れを順に確認します。


## 手順概要
1. GitHub からリポジトリを取得して作業ディレクトリを移動します。
2. PyTorch など必要な Python パッケージをインストールします。
3. サンプル設定で学習 + 解析の一括スクリプトを実行し、結果 JSON を確認します。
4. 研究目的に合わせてハイパーパラメータや解析を変更します。


In [ ]:
#@title リポジトリを取得して作業フォルダに移動する
import os
import subprocess
from pathlib import Path

PROJECT_DIR = Path.cwd() / "world_model_game"

if not PROJECT_DIR.exists():
    repo_url = input("クローンするGitHubリポジトリURLを入力してください (例: https://github.com/your-account/world_model_game.git): ").strip()
    if not repo_url:
        raise RuntimeError("リポジトリURLが指定されていません。上の説明を参照して正しいURLを入力してください。")
    subprocess.run(["git", "clone", repo_url], check=True)
    repo_name = repo_url.rstrip("/").split("/")[-1]
    if repo_name.endswith(".git"):
        repo_name = repo_name[:-4]
    PROJECT_DIR = Path.cwd() / repo_name
    if not PROJECT_DIR.exists():
        raise RuntimeError(f"期待したディレクトリ {repo_name} が見つかりません。クローン先を確認してください。")
else:
    print("既存の world_model_game ディレクトリを使用します。")

os.chdir(PROJECT_DIR)
print(f"作業ディレクトリ: {Path.cwd()}")


In [ ]:
#@title 依存ライブラリをインストールする
import importlib
import subprocess
import sys

def ensure(package: str, pip_target: str | None = None) -> None:
    '''指定したパッケージが無ければ pip でインストールする。'''
    try:
        importlib.import_module(package)
        print(f"{package} は既にインストールされています。")
    except ImportError:
        target = pip_target or package
        print(f"{target} をインストールします...")
        subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", target], check=True)
        importlib.import_module(package)
        print(f"{package} のインストールが完了しました。")

ensure("torch", "torch")
ensure("numpy")


In [ ]:
#@title サンプル設定で学習と解析を実行する
import subprocess
import sys
from pathlib import Path

output_path = Path("experiment_results_sample.json")
cmd = [
    sys.executable,
    "scripts/run_experiment.py",
    "--episodes", "200",
    "--max-steps", "10",
    "--num-agents", "4",
    "--output", str(output_path),
]
print("実行コマンド:", " ".join(cmd))
subprocess.run(cmd, check=True)
print("結果を", output_path, "に保存しました。")


In [ ]:
#@title 生成された結果 JSON を確認する
import json
from pathlib import Path

output_path = Path("experiment_results_sample.json")
if not output_path.exists():
    raise FileNotFoundError("結果ファイルがまだ生成されていません。前のセルを実行してください。")

with output_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

data


## 次のステップ
- `scripts/run_experiment.py` の引数を変更して、エピソード数やエージェント数、タグの種類などを探索してください。
- `world_model_game/analysis.py` の関数を直接呼び出すセルを追加し、隠れ状態プロービングやアテンション分析を個別に試せます。
- 計算資源に余裕がある場合は Colab Pro+ や GPU セッションを利用し、より長いエピソードや大きなモデルを学習させることも可能です。
